In [1]:
import sys
sys.path.append('../')
from setting import config_read

In [2]:
from owlready2 import *
import pandas as pd
from elasticsearch import Elasticsearch
from konlpy.tag import Okt

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation




In [3]:
# Read config file
config = config_read('../')

# Load owl file
data_path = config['owl']['path']
onto = get_ontology(data_path).load()

In [6]:
# Elasticsearch
server_ip = config['elasticsearch']['ip']
index_name = 'wiki'
es = Elasticsearch(server_ip)

In [7]:
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name,ignore=[400, 404])
    print('Index has been deleted successfully')

In [10]:
import rdflib

In [12]:
g = rdflib.Graph()
g.parse('./tbl.ttl')

Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#dateTime, Converter=<function parse_datetime at 0x00000271B319E820>
Traceback (most recent call last):
  File "c:\Users\USER\anaconda3\envs\py39\lib\site-packages\rdflib\term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "c:\Users\USER\anaconda3\envs\py39\lib\site-packages\isodate\isodatetime.py", line 55, in parse_datetime
    tmpdate = parse_date(datestring)
  File "c:\Users\USER\anaconda3\envs\py39\lib\site-packages\isodate\isodates.py", line 203, in parse_date
    raise ISO8601Error('Unrecognised ISO 8601 date format: %r' % datestring)
isodate.isoerror.ISO8601Error: Unrecognised ISO 8601 date format: '-0220-01-01'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#dateTime, Converter=<function parse_datetime at 0x00000271B319E820>
Traceback (most recent call last):
  File "c:\Users\USER\anaconda3\

<Graph identifier=N61e6313de9e541deaf93eb0834f008a9 (<class 'rdflib.graph.Graph'>)>

In [20]:
knows_query = """
SELECT ?instance1 ?p ?instance2 ?class1 ?class2
WHERE {
    ?instance1 ?p ?instance2.
    ?instance1 wdt:P31 ?class1.
    ?instance2 wdt:P31 ?class2.
}"""

qres = g.query(knows_query)

In [21]:
triple_list = []
for row in qres:
    p = str(row[1]).replace('http://www.wikidata.org/prop/direct/', 'wdt:')
    i1 = str(row[0]).replace('http://www.wikidata.org/entity/', 'wd:')
    i2 = str(row[2]).replace('http://www.wikidata.org/entity/', 'wd:')
    c1 = str(row[3]).replace('http://www.wikidata.org/entity/', 'wd:')
    c2 = str(row[4]).replace('http://www.wikidata.org/entity/', 'wd:')

    triple_list.append([i1, p, i2, c1, c2])
len(triple_list)

5669

In [22]:
df = pd.DataFrame(triple_list, columns=['S', 'P', 'O', 'domain', 'range'])
df

,S,P,O,domain,range
0,wd:P2002,wdt:P17,wd:Q30,wd:Q105388954,wd:Q3624078
1,wd:P2002,wdt:P17,wd:Q30,wd:Q105388954,wd:Q6256
2,wd:P2002,wdt:P17,wd:Q30,wd:Q105388954,wd:Q1520223
3,wd:P2002,wdt:P17,wd:Q30,wd:Q105388954,wd:Q512187
4,wd:P2002,wdt:P17,wd:Q30,wd:Q105388954,wd:Q5255892
...,...,...,...,...,...
5664,wd:Q30,wdt:P463,wd:Q1065,wd:Q1489259,wd:Q484652
5665,wd:Q30,wdt:P31,wd:Q6256,wd:Q1489259,wd:Q104098715
5666,wd:Q30,wdt:P31,wd:Q6256,wd:Q1489259,wd:Q15617994
5667,wd:Q30,wdt:P2936,wd:Q1860,wd:Q1489259,wd:Q34770


In [30]:
df[['S', 'domain']].drop_duplicates()

,S,domain
0,wd:P2002,wd:Q105388954
6,wd:P2002,wd:Q24239898
12,wd:P2002,wd:Q62589320
18,wd:P2002,wd:Q99601935
24,wd:Q312,wd:Q43229
...,...,...
5596,wd:Q6581097,wd:Q48264
5598,wd:Q145,wd:Q1250464
5611,wd:Q145,wd:Q20181813
5624,wd:Q1860,wd:Q34770


In [43]:
temp_df = df[['O', 'range']].drop_duplicates()
temp_df.columns = ['S', 'domain']
ins_cls = pd.concat([df[['S', 'domain']].drop_duplicates(), temp_df]).drop_duplicates()
ins_cls = ins_cls.groupby('S')['domain'].apply(list).reset_index()

In [55]:
import json
with open("../qald10_id2label.json", "r") as f:
    id2label = json.load(f)

In [57]:
for i, row in ins_cls.iterrows():

    if row[0] in id2label.keys():
        # print(row[0], row[1], id2label[row[0]])
        doc = {'URI':row[0], 'Type':'T_i', 'Annotation Values':id2label[row[0]], 'Tbox':row[1]}
        es.index(index=index_name, body=doc)
        # break

    # doc = {'URI':row[0], 'Type':'T_i', 'Annotation Values':row[2]}
    # es.index(index=index_name, body=doc)

In [58]:
es.search(index=['wiki'], )

{'took': 232,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 328, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'wiki',
    '_id': 'gHcI_IsBvRqqBWK5NcUX',
    '_score': 1.0,
    '_source': {'URI': 'wd:Q1011547',
     'Type': 'T_i',
     'Annotation Values': 'Golden Globe Award',
     'Tbox': ['wd:Q107655869']}},
   {'_index': 'wiki',
    '_id': 'gXcI_IsBvRqqBWK5PcUg',
    '_score': 1.0,
    '_source': {'URI': 'wd:Q103618',
     'Type': 'T_i',
     'Annotation Values': 'Academy Award for Best Actress',
     'Tbox': ['wd:Q19020', 'wd:Q96474676']}},
   {'_index': 'wiki',
    '_id': 'gncI_IsBvRqqBWK5PcVM',
    '_score': 1.0,
    '_source': {'URI': 'wd:Q1048',
     'Type': 'T_i',
     'Annotation Values': 'Julius Caesar',
     'Tbox': ['wd:Q5']}},
   {'_index': 'wiki',
    '_id': 'g3cI_IsBvRqqBWK5PcVg',
    '_score': 1.0,
    '_source': {'URI': 'wd:Q1055',
     'Type': 'T_i',
     'Annotation Values': 'H